# Pregunta 1

In [ ]:
# Get absolute distance between letters on a string and an alphabet
def abs_distance(string, frequencies):
    return sum([abs(frequencies[c] - string.count(c) / len(string)) for c in frequencies])

In [ ]:
def break_rp(ciphertext, frequencies, distance):
    """
    Arguments:
        ciphertext: An arbitrary string representing the encrypted version of a plaintext.
        frequencies: A dictionary representing a character frequency over the alphabet.
        distance: A function indicating how distant is a string from following a character frequency.
    Returns:
        key: A guess of the key used to encrypt the ciphertext, assuming that the plaintext message was written in a language in which
        letters distribute according to frequencies.
    """
    return 'hola1234'